In [9]:
# LSTM + CNN
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Conv1D
from tensorflow.keras.callbacks import EarlyStopping
from pyproj import Transformer

import binascii
from shapely import wkb
import pyproj

def convert_wkb_to_coordinates(wkb_string):
    
    # 0단계 :Check if the value is a float
    if isinstance(wkb_string, float):
        return None, None
    
    # 1단계: 16진수 문자열을 이진 형식으로 디코딩
    binary_string = binascii.unhexlify(wkb_string)

    # 2단계: 이진 형식 파싱 
    geometry = wkb.loads(binary_string)

    # 3단계: 좌표를 위도와 경도로 변환
    if geometry and hasattr(geometry, 'wkt'):
        # Well-Known Text (WKT) 형식으로 변환
        wkt = geometry.wkt

        # 좌표 시스템 변환 정의
        #!crs_from = pyproj.Proj(init='epsg:3857')
        #!crs_to = pyproj.Proj(init='epsg:4326')
        transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
        transformer.transform(12, 12)

        # 좌표 변환 수행
        #! transformed = pyproj.transform(crs_from, crs_to, geometry.x, geometry.y)
        transformed = Transformer.transform(transformer,geometry.x, geometry.y)

        # 위도와 경도 추출
        latitude, longitude = transformed[0], transformed[1]
        
        return latitude, longitude
    else:
        return None, None
    

# CSV 파일에서 데이터 로드
data = pd.read_csv('D:/장우영/LOCALSEARCH/Ship_DA/DA/data/FAmerge_20230607_114714.csv', encoding='ANSI')

# 특정 칼럼의 조건에 따라 데이터 필터링
filtered_data = data[data['sog'] > 3]  # 특정 칼럼과 조건을 적절히 수정해야 합니다

data= filtered_data


# 좌표 변환 적용 및 데이터프레임 업데이트
data[['latitude', 'longitude']] = data['geom'].apply(convert_wkb_to_coordinates).apply(pd.Series)

#print(data[['latitude', 'longitude']])

# "insert_time"을 숫자로 변환
data['year'] = pd.to_datetime(data['insert_time']).dt.year
data['month'] = pd.to_datetime(data['insert_time']).dt.month
data['day'] = pd.to_datetime(data['insert_time']).dt.day
data['hour'] = pd.to_datetime(data['insert_time']).dt.hour
data['minute'] = pd.to_datetime(data['insert_time']).dt.minute
data['second'] = pd.to_datetime(data['insert_time']).dt.minute


# 특성과 타겟 변수 선택
X = data[["mmsi", "ship_type", "latitude", "longitude", "cog", "sog", "year", "month", "day", "hour", "minute", "second", "풍향", "유향", "기온", "수온", "풍속", "유속", "기압", "습도"]]
y = data[["latitude", "longitude","cog", "sog"]]

print(X)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MinMaxScaler를 사용하여 입력 특성 스케일 조정
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# LSTM에 맞게 입력 데이터 형태 변환
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))


# LSTM과 CNN을 결합한 모델 생성
model = Sequential()

# Conv1D 레이어를 추가합니다. 1D 컨볼루션 연산을 수행하는 레이어로, 입력 데이터의 필터 수는 64개, 커널 크기는 3입니다. 활성화 함수로는 ReLU를 사용하고, 입력 형태는 X_train_reshaped의 shape[1]과 shape[2]에 해당합니다.
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))

#  LSTM은 순환 신경망의 한 종류로, 입력 데이터에 대한 장기 의존성을 모델링하는 데 사용됩니다. 이 레이어의 뉴런 수는 64개    
model.add(LSTM(64))

# Dense 레이어를 추가합니다. 출력층으로 사용되며, 뉴런 수는 4개입니다. 이 뉴런은 (latitude, longitude, sog, cog)와 같은 4개의 출력 값을 예측하는 데 사용
model.add(Dense(4))  

# 모델 컴파일
model.compile(loss='mse', optimizer='adam')

# 조기 종료를 위한 EarlyStopping 정의
#early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

# 모델 훈련
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_test))

# 모델을 사용하여 예측 수행
y_pred = model.predict(X_test_reshaped)

# Convert predictions to a list
y_pred_list = y_pred.tolist()

# 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Create the response JSON
response = {
    "latitude": y_pred_list[0][0],
    "longitude": y_pred_list[0][1],
    "predicted_cog": y_pred_list[0][2],
    "predicted_sog": y_pred_list[0][3]
}

print(response)

            mmsi  ship_type   latitude   longitude    cog   sog  year  month   
0      440051540          0  35.039909  129.062547  329.2   5.7  2023      5  \
5      538010219          0  34.882612  129.060553  212.2  16.1  2023      5   
17     440051540          0  35.039909  129.062547  329.2   5.7  2023      5   
19     440102990          0  35.099860  129.041436   42.6   8.8  2023      5   
20     440048210         80  35.064618  129.110167   31.6   9.6  2023      5   
...          ...        ...        ...         ...    ...   ...   ...    ...   
52705  440135680          0  34.958493  129.193000  306.5  10.1  2023      5   
52708  440658000         70  35.042145  129.170300  239.4   7.5  2023      5   
52709  440761000         71  35.019416  129.126246  210.0   7.6  2023      5   
52714  440003510          0  35.035548  129.038131   90.6  15.3  2023      5   
52726  563156900         71  34.813431  129.000033   49.5  11.8  2023      5   

       day  hour  minute  second   풍향  

In [2]:
# LSTM + CNN
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Conv1D
from tensorflow.keras.callbacks import EarlyStopping
from pyproj import Transformer

import binascii
from shapely import wkb
import pyproj

def convert_wkb_to_coordinates(wkb_string):
    
    # 0단계 :Check if the value is a float
    if isinstance(wkb_string, float):
        return None, None
    
    # 1단계: 16진수 문자열을 이진 형식으로 디코딩
    binary_string = binascii.unhexlify(wkb_string)

    # 2단계: 이진 형식 파싱 
    geometry = wkb.loads(binary_string)

    # 3단계: 좌표를 위도와 경도로 변환
    if geometry and hasattr(geometry, 'wkt'):
        # Well-Known Text (WKT) 형식으로 변환
        wkt = geometry.wkt

        # 좌표 시스템 변환 정의
        #!crs_from = pyproj.Proj(init='epsg:3857')
        #!crs_to = pyproj.Proj(init='epsg:4326')
        transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
        transformer.transform(12, 12)

        # 좌표 변환 수행
        #! transformed = pyproj.transform(crs_from, crs_to, geometry.x, geometry.y)
        transformed = Transformer.transform(transformer,geometry.x, geometry.y)

        # 위도와 경도 추출
        latitude, longitude = transformed[0], transformed[1]
        
        return latitude, longitude
    else:
        return None, None
    

# CSV 파일에서 데이터 로드
data = pd.read_csv('D:/장우영/LOCALSEARCH/Ship_DA/DA/data/FAmerge_20230607_114714.csv', encoding='ANSI')

# 특정 칼럼의 조건에 따라 데이터 필터링
filtered_data = data[data['sog'] > 3]  # 특정 칼럼과 조건을 적절히 수정해야 합니다

data= filtered_data


# 좌표 변환 적용 및 데이터프레임 업데이트
data[['latitude', 'longitude']] = data['geom'].apply(convert_wkb_to_coordinates).apply(pd.Series)

#print(data[['latitude', 'longitude']])

# "insert_time"을 숫자로 변환
data['year'] = pd.to_datetime(data['insert_time']).dt.year
data['month'] = pd.to_datetime(data['insert_time']).dt.month
data['day'] = pd.to_datetime(data['insert_time']).dt.day
data['hour'] = pd.to_datetime(data['insert_time']).dt.hour
data['minute'] = pd.to_datetime(data['insert_time']).dt.minute
data['second'] = pd.to_datetime(data['insert_time']).dt.minute


# 특성과 타겟 변수 선택
X = data[["mmsi", "ship_type", "latitude", "longitude", "cog", "sog", "year", "month", "day", "hour", "minute", "second", "풍향", "유향", "기온", "수온", "풍속", "유속", "기압", "습도"]]

# 각 변수에 대한 타겟 변수 선택
y_latitude = data[["latitude"]]
y_longitude = data[["longitude"]]
y_cog = data[["cog"]]
y_sog = data[["sog"]]

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_latitude_train, y_latitude_test, y_longitude_train, y_longitude_test, y_cog_train, y_cog_test, y_sog_train, y_sog_test = train_test_split(X, y_latitude, y_longitude, y_cog, y_sog, test_size=0.2, random_state=42)

# MinMaxScaler를 사용하여 입력 특성 스케일 조정
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# LSTM에 맞게 입력 데이터 형태 변환
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# LSTM과 CNN을 결합한 모델 생성
model_latitude = Sequential()
model_longitude = Sequential()
model_cog = Sequential()
model_sog = Sequential()

# Conv1D 레이어를 추가합니다. 1D 컨볼루션 연산을 수행하는 레이어로, 입력 데이터의 필터 수는 64개, 커널 크기는 3입니다. 활성화 함수로는 ReLU를 사용하고, 입력 형태는 X_train_reshaped의 shape[1]과 shape[2]에 해당합니다.
model_latitude.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model_longitude.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model_cog.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model_sog.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))

# LSTM은 순환 신경망의 한 종류로, 입력 데이터에 대한 장기 의존성을 모델링하는 데 사용됩니다. 이 레이어의 뉴런 수는 64개
model_latitude.add(LSTM(64))
model_longitude.add(LSTM(64))
model_cog.add(LSTM(64))
model_sog.add(LSTM(64))

# Dense 레이어를 추가합니다. 출력층으로 사용되며, 뉴런 수는 1개입니다. 각각의 모델에 대해 예측을 수행하므로 출력은 1개
model_latitude.add(Dense(1))
model_longitude.add(Dense(1))
model_cog.add(Dense(1))
model_sog.add(Dense(1))

# 모델 컴파일
model_latitude.compile(loss='mse', optimizer='adam')
model_longitude.compile(loss='mse', optimizer='adam')
model_cog.compile(loss='mse', optimizer='adam')
model_sog.compile(loss='mse', optimizer='adam')

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='min')


# 모델 훈련
history_latitude = model_latitude.fit(X_train_reshaped, y_latitude_train, epochs=30, batch_size=32, validation_data=(X_test_reshaped, y_latitude_test))
history_longitude = model_longitude.fit(X_train_reshaped, y_longitude_train, epochs=30, batch_size=32, validation_data=(X_test_reshaped, y_longitude_test))
history_cog = model_cog.fit(X_train_reshaped, y_cog_train, epochs=30, batch_size=32, validation_data=(X_test_reshaped, y_cog_test))
history_sog = model_sog.fit(X_train_reshaped, y_sog_train, epochs=30, batch_size=32, validation_data=(X_test_reshaped, y_sog_test))

# 모델을 사용하여 예측 수행
y_latitude_pred = model_latitude.predict(X_test_reshaped)
y_longitude_pred = model_longitude.predict(X_test_reshaped)
y_cog_pred = model_cog.predict(X_test_reshaped)
y_sog_pred = model_sog.predict(X_test_reshaped)

# Convert predictions to a list
y_latitude_pred_list = y_latitude_pred.tolist()
y_longitude_pred_list = y_longitude_pred.tolist()
y_cog_pred_list = y_cog_pred.tolist()
y_sog_pred_list = y_sog_pred.tolist()

# 평균 제곱 오차 계산
mse_latitude = mean_squared_error(y_latitude_test, y_latitude_pred)
mse_longitude = mean_squared_error(y_longitude_test, y_longitude_pred)
mse_cog = mean_squared_error(y_cog_test, y_cog_pred)
mse_sog = mean_squared_error(y_sog_test, y_sog_pred)

print("Mean Squared Error - Latitude:", mse_latitude)
print("Mean Squared Error - Longitude:", mse_longitude)
print("Mean Squared Error - COG:", mse_cog)
print("Mean Squared Error - SOG:", mse_sog)

# Create the response JSON
response = {
    "latitude": y_latitude_pred_list[0][0],
    "longitude": y_longitude_pred_list[0][0],
    "predicted_cog": y_cog_pred_list[0][0],
    "predicted_sog": y_sog_pred_list[0][0]
}

print(response)

Epoch 1/100
3842/3842 [==============================] - 21s 5ms/step - loss: 26.6726 - val_loss: 0.0634
Epoch 2/100
3842/3842 [==============================] - 20s 5ms/step - loss: 0.0601 - val_loss: 0.0628
Epoch 3/100
3842/3842 [==============================] - 22s 6ms/step - loss: 0.0587 - val_loss: 0.0619
Epoch 4/100
3842/3842 [==============================] - 22s 6ms/step - loss: 0.0588 - val_loss: 0.0610
Epoch 5/100
3842/3842 [==============================] - 22s 6ms/step - loss: 0.0587 - val_loss: 0.0609
Epoch 6/100
3842/3842 [==============================] - 22s 6ms/step - loss: 0.0587 - val_loss: 0.0622
Epoch 7/100
3842/3842 [==============================] - 21s 5ms/step - loss: 0.0586 - val_loss: 0.0616
Epoch 8/100
3842/3842 [==============================] - 21s 6ms/step - loss: 0.0586 - val_loss: 0.0635
Epoch 9/100
3842/3842 [==============================] - 21s 5ms/step - loss: 0.0586 - val_loss: 0.0619
Epoch 10/100
3842/3842 [==============================] - 22s 6

Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_18112\1820761606.py", line 132, in <module>
    history_sog = model_sog.fit(X_train_reshaped, y_sog_train, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_sog_test))
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1685, in fit
    tmp_logs = self.train_function(iterator)
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\SW\AppData\Local\Progra

In [5]:
import pickle

# 모델 저장
with open('model_latitude02.pkl', 'wb') as f:
    pickle.dump(model_latitude, f)

with open('model_longitude02.pkl', 'wb') as f:
    pickle.dump(model_longitude, f)

with open('model_cog02.pkl', 'wb') as f:
    pickle.dump(model_cog, f)

with open('model_sog02.pkl', 'wb') as f:
    pickle.dump(model_sog, f)

# 모델 불러오기
with open('model_latitude02.pkl', 'rb') as f:
    model_latitude = pickle.load(f)

with open('model_longitude02.pkl', 'rb') as f:
    model_longitude = pickle.load(f)

with open('model_cog02.pkl', 'rb') as f:
    model_cog = pickle.load(f)

with open('model_sog02.pkl', 'rb') as f:
    model_sog = pickle.load(f)
